In [ ]:
# base model for the training process, external testing process, and drug stratified analysis
library(survival)
library(nlme)
library(splines)
library(JMbayes2)
# replace ".." with the training, external testing, and drug stratified dataset as needed
df_final_surv <- read.csv("..")
df_final_long <- read.csv("..")

CoxFit3 <- coxph(Surv(time1, IIR_3revi) ~ B9 + B19_group + AGE_10, data = df_final_surv)
summary(CoxFit3)

lmeFit1 <- lme(F2 ~ ns(follow.up.years, 2, B = c(0, 6.7)) + B9 + B8 + B12_group + B18_group + B10_group + AGE_10 + B16_group, 
               data = df_final_long, random = ~ ns(follow.up.years, 2, B = c(0, 6.7)) | NID,
               control = lmeControl(opt = "optim"))

lmeFit2 <- lme(CD4CD8 ~ ns(follow.up.years, 2, B = c(0, 6.4)) + B2 + AGE_10 + B25 + B12_group + B16_group + B9 + B8 + B10_group, 
               data = df_final_long, random = ~ ns(follow.up.years, 2, B = c(0, 6.4)) | NID,
               control = lmeControl(opt = "optim"))

jmFitcox3.12 <- jm(CoxFit3, list(lmeFit1, lmeFit2), time_var = "follow.up.years")

In [ ]:
# model ROAUC
library(JMbayes2)
library(timeROC)

jmFitcox3.12_roc <- tvROC(jmFitcox3.12, newdata = df_final_long, Tstart = .., Thoriz = ..)
# Tstart = 1, 2, 3, 4; Thoriz = 5
# Tstart = 1, 2, 3, 4, 5; Thoriz = 6
# Tstart = 1, 2, 3, 4, 5, 6; Thoriz = 7
print(tvAUC(jmFitcox3.12_roc))


In [ ]:
# model calibration
library(JMbayes2)
library(ggplot2)

jmFitcox3.12_cp <- calibration_plot(jmFitcox3.12, newdata = df_final_long, Tstart = .., Thoriz = ..)
# Tstart = 1, 2, 3, 4; Thoriz = 5
# Tstart = 1, 2, 3, 4, 5; Thoriz = 6
# Tstart = 1, 2, 3, 4, 5, 6; Thoriz = 7

In [ ]:
# base model for internal validation
library(splitstackshape)
library(dplyr)
library(survival)
library(nlme)
library(splines)
library(JMbayes2)
library(timeROC)

df_total <- data.frame()

for (i in 1:50) {
  bootstrap_index <- as.integer(rownames(stratified(df_final_surv, "IIR_3revi", size = nrow(df_final_surv) * 0.5)))
  
  valid_data_surv <- df_final_surv[bootstrap_index, ]
  valid_data_long <- dplyr::filter(df_final_long, NID %in% valid_data_surv$NID)
  valid_data_surv <- dplyr::filter(valid_data_surv, NID %in% valid_data_long$NID)
  # Tstart = 1, 2, 3, 4, 5; Thoriz = 6
  CoxFit3 <- coxph(Surv(time1, IIR_3revi) ~ B9+B19_group+BMI_group+AGE_10, data = valid_data_surv)
  lmeFit1 <- lme(F2 ~ ns(follow.up.years, 2, B = c(0, 6.7))+B9+B8+B12_group+BMI_group+B18_group+B10_group+AGE_10+B16_group, 
                 data = valid_data_long, random = ~ ns(follow.up.years, 2, B = c(0, 6.7)) | NID,
                 control = lmeControl(opt = "optim"))
  lmeFit2 <- lme(CD4CD8 ~ ns(follow.up.years, 2, B = c(0, 6.4))+B2+AGE_10+B25+B12_group+B16_group+B9+B8+B10_group, 
                 data = valid_data_long, random = ~ ns(follow.up.years, 2, B = c(0, 6.4)) | NID,
                 control = lmeControl(opt = "optim"))
  jmFitcox3.12 <- jm(CoxFit3, list(lmeFit1, lmeFit2), time_var = "follow.up.years")
  
  roc_result16 <- tvROC(jmFitcox3.12, newdata = valid_data_long, Tstart = 1, Thoriz = 6)
  roc_result26 <- tvROC(jmFitcox3.12, newdata = valid_data_long, Tstart = 2, Thoriz = 6)
  roc_result36 <- tvROC(jmFitcox3.12, newdata = valid_data_long, Tstart = 3, Thoriz = 6)
  roc_result46 <- tvROC(jmFitcox3.12, newdata = valid_data_long, Tstart = 4, Thoriz = 6)
  roc_result56 <- tvROC(jmFitcox3.12, newdata = valid_data_long, Tstart = 5, Thoriz = 6)
  
  result1 <- tvAUC(roc_result16)
  result2 <- tvAUC(roc_result26)
  result3 <- tvAUC(roc_result36)
  result4 <- tvAUC(roc_result46)
  result5 <- tvAUC(roc_result56)

  df1 <- data.frame(
  Estimated_AUC = result1$auc,
  At_time = result1$Tstart,
  Subjects = result1$nr
)
  df2 <- data.frame(
  Estimated_AUC = result2$auc,
  At_time = result2$Tstart,
  Subjects = result2$nr
)
  df3 <- data.frame(
  Estimated_AUC = result3$auc,
  At_time = result3$Tstart,
  Subjects = result3$nr
)
  df4 <- data.frame(
  Estimated_AUC = result4$auc,
  At_time = result4$Tstart,
  Subjects = result4$nr
)
  df5 <- data.frame(
  Estimated_AUC = result5$auc,
  At_time = result5$Tstart,
  Subjects = result5$nr
)
  df_total <- rbind(df_total, df1, df2, df3, df4, df5)
}

In [119]:
# update models
library(JMbayes2)

jmFitcox3.12.value <- update(jmFitcox3.12, functional_forms = ~ value(F2) + value(CD4CD8))
jmFitcox3.12.slope <- update(jmFitcox3.12, functional_forms = ~ slope(F2) + slope(CD4CD8))
jmFitcox3.12.slope1 <- update(jmFitcox3.12, functional_forms = ~ value(F2) + slope(F2) 
                             + value(CD4CD8) + slope(CD4CD8))
jmFitcox3.12.area <- update(jmFitcox3.12, functional_forms = ~ area(F2) + area(CD4CD8))
jmFitcox3.12.area1 <- update(jmFitcox3.12, functional_forms = ~ value(F2) + area(F2) 
                         + value(CD4CD8) + area(CD4CD8))

In [22]:
# model updated by adding cross-sectional values at 4-year follow-up
library(survival)
library(nlme)
library(splines)
library(JMbayes2)

df_surv_4 <- read.csv("..")
df_long_4 <- read.csv("..")

CoxFit3 <- coxph(Surv(time1, IIR_3revi) ~ B9+B19_group+BMI_group+AGE_10, data = df_surv_4)
lmeFit1 <- lme(F2 ~ ns(follow.up.years, 2, B = c(0, 6.7))+B9+B8+B12_group+BMI_group+B18_group
                +B10_group+AGE_10+B16_group+F2_4+CD4CD8_4, 
                data = df_long_4, random = ~ ns(follow.up.years, 2, B = c(0, 6.7)) | NID,
                control = lmeControl(opt = "optim"))
lmeFit2 <- lme(CD4CD8 ~ ns(follow.up.years, 2, B = c(0, 6.4))+B2+AGE_10+B25+B12_group+B16_group+B9
                +B8+B10_group+F2_4+CD4CD8_4, 
                data = df_long_4, random = ~ ns(follow.up.years, 2, B = c(0, 6.4)) | NID,
                control = lmeControl(opt = "optim"))
jmFitcox3.12 <- jm(CoxFit3, list(lmeFit1, lmeFit2), time_var = "follow.up.years")